In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
%config IPCompleter.greedy=True

## UPLOAD DATASETS

In [49]:
path = '../../'

In [50]:
name_cat = 'cat_results_eda.pickle'
name_num = 'num_results_eda.pickle'

In [51]:
cat = pd.read_pickle(path + '/02_Data/03_Work/' + name_cat)
num = pd.read_pickle(path + '/02_Data/03_Work/' + name_num)

## NEW VARIABLES

In this case, they are already created for us:

* date components
* calendar variables

to create:

* those that we identify in EDA
* lags
* mobile windows

For this phase we need to rejoin into a single dataframe.

In [52]:
df = pd.concat([cat,num], axis = 1)
df

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,sales,sell_price
date,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.25
2013-01-01,CA_3,FOODS_3_120,d_704,2013,1,4,Tuesday,NewYear,National,11249,33,1.25
2013-01-01,CA_3,FOODS_3_202,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,FOODS_3_252,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,4.98
2013-01-01,CA_3,FOODS_3_288,d_704,2013,1,4,Tuesday,NewYear,National,11249,20,4.28
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-30,CA_4,FOODS_3_329,d_1767,2015,11,3,Monday,No_event,No_event,11544,9,1.68
2015-11-30,CA_4,FOODS_3_555,d_1767,2015,11,3,Monday,No_event,No_event,11544,26,2.48
2015-11-30,CA_4,FOODS_3_586,d_1767,2015,11,3,Monday,No_event,No_event,11544,13,2.48


In [53]:
df.item_id.value_counts()

FOODS_3_090    2128
FOODS_3_120    2128
FOODS_3_202    2128
FOODS_3_252    2128
FOODS_3_288    2128
FOODS_3_329    2128
FOODS_3_555    2128
FOODS_3_586    2128
FOODS_3_587    2128
FOODS_3_714    2128
Name: item_id, dtype: int64

### Variable intermittent demand

This variable will identify how many consecutive days have elapsed with zero sales.

We will define it as if the last n days have had zero sales then there is a stock out.

We can create several by changing the n.

It will help us to model.

In [54]:
def stock_break(sales, n = 5):
    zero_sales = pd.Series(np.where(sales == 0,1,0))
    num_zeros = zero_sales.rolling(n).sum()
    stock_break = np.where(num_zeros == n,1,0)
    return(stock_break)

In [55]:
df = df.sort_values(by = ['store_id','item_id','date'])

In [56]:
df['stock_break_3'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x, 3)).values

In [57]:
df['stock_break_7'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x,7)).values

In [58]:
df['stock_break_15'] = df.groupby(['store_id','item_id']).sales.transform(lambda x: stock_break(x,15)).values

### lag Variables

create lags on the following variables:

* sales: lags of 15 days
* sell_price: 7 day lags
* break_stock: lag of one day

In [59]:
def make_lags(df, variable, num_lags = 7):
    lags = pd.DataFrame()
    
    #make lags
    for cada in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(cada)] = df[variable].shift(cada)

    return(lags)

In [60]:
lags_sell_price_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: make_lags(df = x, variable = 'sell_price', num_lags= 7))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\1798082905.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: make_lags(df = x, variable = 'sell_price', num_lags= 7))


In [61]:
lags_stock_break_3_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: make_lags(df = x, variable = 'stock_break_3', num_lags= 1))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\449861860.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: make_lags(df = x, variable = 'stock_break_3', num_lags= 1))


In [62]:
lags_stock_break_7_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: make_lags(df = x, variable = 'stock_break_7', num_lags= 1))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\1882374383.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: make_lags(df = x, variable = 'stock_break_7', num_lags= 1))


In [63]:
lags_stock_break_15_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: make_lags(df = x, variable = 'stock_break_15', num_lags= 1))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\602735986.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: make_lags(df = x, variable = 'stock_break_15', num_lags= 1))


In [64]:
lags_sales_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: make_lags(df = x, variable = 'sales', num_lags= 15))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\2504107476.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: make_lags(df = x, variable = 'sales', num_lags= 15))


In [65]:
df

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,sales,sell_price,stock_break_3,stock_break_7,stock_break_15
date,,,,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,0,1.25,0,0,0
2013-01-02,CA_3,FOODS_3_090,d_705,2013,1,5,Wednesday,No_event,No_event,11249,224,1.25,0,0,0
2013-01-03,CA_3,FOODS_3_090,d_706,2013,1,6,Thursday,No_event,No_event,11249,241,1.25,0,0,0
2013-01-04,CA_3,FOODS_3_090,d_707,2013,1,7,Friday,No_event,No_event,11249,232,1.25,0,0,0
2013-01-05,CA_3,FOODS_3_090,d_708,2013,1,1,Saturday,No_event,No_event,11250,301,1.25,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-26,CA_4,FOODS_3_714,d_1763,2015,11,6,Thursday,Thanksgiving,National,11543,22,1.58,0,0,0
2015-11-27,CA_4,FOODS_3_714,d_1764,2015,11,7,Friday,No_event,No_event,11543,10,1.58,0,0,0
2015-11-28,CA_4,FOODS_3_714,d_1765,2015,11,1,Saturday,No_event,No_event,11544,17,1.58,0,0,0


### Mobile window variables

create three types of mobile windows about sales:

* mobile minimum
* moving average
* moving maximum

Each of them in the range of 15 days.

In [66]:
def min_movil(df, variable, num_periods = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periods+1):
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()

    return(minm)

In [67]:
def mean_movil(df, variable, num_periods = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periods+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()
    
    return(mm)

In [68]:
def max_movil(df, variable, num_periods = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periods+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()

    return(maxm)

In [69]:
min_movil_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: min_movil(df = x, variable = 'sales', num_periods= 15))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\2637024586.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: min_movil(df = x, variable = 'sales', num_periods= 15))


In [70]:
mean_movil_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: mean_movil(df = x, variable = 'sales', num_periods= 15))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\403060666.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: mean_movil(df = x, variable = 'sales', num_periods= 15))


In [71]:
max_movil_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: max_movil(df = x, variable = 'sales', num_periods= 15))

C:\Users\luise\AppData\Local\Temp\ipykernel_1996\3725088011.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: max_movil(df = x, variable = 'sales', num_periods= 15))


## PREPARE  DATASETS

### Join all generated dataframes

In [72]:
df_join = pd.concat([df,
                      lags_sell_price_df,
                      lags_stock_break_3_df,
                      lags_stock_break_7_df,
                      lags_stock_break_15_df,
                      lags_sales_df,
                      min_movil_df,
                      mean_movil_df,
                      max_movil_df], axis = 1)

df_join

,store_id,item_id,d,year,month,wday,weekday,event_name_1,event_type_1,wm_yr_wk,...,sales_maxm_6,sales_maxm_7,sales_maxm_8,sales_maxm_9,sales_maxm_10,sales_maxm_11,sales_maxm_12,sales_maxm_13,sales_maxm_14,sales_maxm_15
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,CA_3,FOODS_3_090,d_704,2013,1,4,Tuesday,NewYear,National,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,CA_3,FOODS_3_090,d_705,2013,1,5,Wednesday,No_event,No_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,CA_3,FOODS_3_090,d_706,2013,1,6,Thursday,No_event,No_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,CA_3,FOODS_3_090,d_707,2013,1,7,Friday,No_event,No_event,11249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,CA_3,FOODS_3_090,d_708,2013,1,1,Saturday,No_event,No_event,11250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-11-26,CA_4,FOODS_3_714,d_1763,2015,11,6,Thursday,Thanksgiving,National,11543,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
2015-11-27,CA_4,FOODS_3_714,d_1764,2015,11,7,Friday,No_event,No_event,11543,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
2015-11-28,CA_4,FOODS_3_714,d_1765,2015,11,1,Saturday,No_event,No_event,11544,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


In [73]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21280 entries, 2013-01-01 to 2015-11-30
Data columns (total 82 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   store_id              21280 non-null  object 
 1   item_id               21280 non-null  object 
 2   d                     21280 non-null  object 
 3   year                  21280 non-null  object 
 4   month                 21280 non-null  object 
 5   wday                  21280 non-null  object 
 6   weekday               21280 non-null  object 
 7   event_name_1          21280 non-null  object 
 8   event_type_1          21280 non-null  object 
 9   wm_yr_wk              21280 non-null  int64  
 10  sales                 21280 non-null  int64  
 11  sell_price            21280 non-null  float64
 12  stock_break_3         21280 non-null  int32  
 13  stock_break_7         21280 non-null  int32  
 14  stock_break_15        21280 non-null  int32  
 15  se

### Remove the nulls that have generated the new variables

In [74]:
df_join.dropna(inplace=True)

### Eliminate the variables that we are not going to need to model

In [77]:
to_drop = ['d','wm_yr_wk','sell_price','stock_break_3','stock_break_7','stock_break_15']

In [78]:
df_join.drop(columns=to_drop, inplace=True)

### target

In [81]:
target = df_join.sales

### Separate  num y cat

In [83]:
cat = df_join.select_dtypes(include='O')

In [84]:
num = df_join.select_dtypes(exclude='O')

## CATEGÓRICAL TRANSFORMATION

### One Hot Encoding

In [85]:
var_ohe = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

In [86]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Fit and Apply

In [87]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

C:\Users\luise\anaconda3\envs\retail\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


#### Save as dataframe

In [88]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Target Encoding

In [89]:
var_te = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Fit and Apply

In [90]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

C:\Users\luise\anaconda3\envs\retail\lib\site-packages\category_encoders\target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [91]:
cat_te = te.fit_transform(cat[var_te], y = target)

#### Save as dataframe

In [93]:
#Add sufix to the names
names_te = [variable + '_te' for variable in var_te]

#Save as dataframe
cat_te = pd.DataFrame(cat_te, columns = names_te)

## JOIN TRANSFORMS DATASETS 

We rescue from df_unido the segmentation variables.

In [94]:
de_df_join = df_join[['store_id','item_id']].reset_index()

de_df_join.head(2)

,date,store_id,item_id
0,2013-01-16,CA_3,FOODS_3_090
1,2013-01-17,CA_3,FOODS_3_090


### Join dataframes

In [95]:
dataframes = [de_df_join, cat_ohe,cat_te,num.reset_index(drop=True)]

In [96]:
df_tablon = pd.concat(dataframes, axis = 1)

df_tablon

,date,store_id,item_id,year_2013,year_2014,year_2015,month_1,month_2,month_3,month_4,...,sales_maxm_6,sales_maxm_7,sales_maxm_8,sales_maxm_9,sales_maxm_10,sales_maxm_11,sales_maxm_12,sales_maxm_13,sales_maxm_14,sales_maxm_15
0,2013-01-16,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
1,2013-01-17,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
2,2013-01-18,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
3,2013-01-19,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,281.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0,351.0
4,2013-01-20,CA_3,FOODS_3_090,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0,362.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20975,2015-11-26,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,17.0,17.0
20976,2015-11-27,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20977,2015-11-28,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
20978,2015-11-29,CA_4,FOODS_3_714,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0


## SAVE DATASETS AFTER DATA TRANSFORMATION

In [97]:
path_df_tablon = path + '/02_Data/03_Work/' + 'df_tablon.pickle'

In [98]:
df_tablon.to_pickle(path_df_tablon)